In [2]:
import gzip
import re
import numpy
from collections import defaultdict
import random
import nltk
from nltk.stem.porter import *
from sklearn import linear_model
from nltk.corpus import stopwords
import string
import operator
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer
from keras.models import Sequential
from keras.layers import Dense, Activation

Using Theano backend.


In [3]:
def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [4]:
# Splitting data into training and validation

training_data = []
validation_data = []
data = []
count = 0

for d in readGz('train.json.gz'):
    if count < 170000:
        if not d['helpful']['outOf'] == 0:
            training_data.append(d)
    else:
        if not d['helpful']['outOf'] == 0:
            validation_data.append(d)
    count += 1
    
stop_words = stopwords.words('english')

In [5]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
stemmer = PorterStemmer()
for d in training_data:
    r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
    for w in r.split():
        w = stemmer.stem(w)
        wordCount[w] += 1

In [6]:
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

In [81]:
'''wordCount2 = defaultdict(int)
punctuation = set(string.punctuation)
stemmer = PorterStemmer()
for d in training_data:
    r = ''.join([c for c in d['summary'].lower() if not c in punctuation])
    for w in r.split():
        if w in stop_words:
            continue
        w = stemmer.stem(w)
        if w in stop_words:
            continue
        wordCount2[w] += 1'''

In [82]:
'''counts = [(wordCount2[w], w) for w in wordCount2]
counts.sort()
counts.reverse()

words2 = [x[1] for x in counts[:1000]]'''

In [95]:
'''def feature(d):
    feat = [1]  # remove this when you train for neural network
    feat.append(d['rating'])
    if 'price' in d:
        feat.append(d['price'])
    else:
        feat.append(-1)
    feat.append(d['unixReviewTime'])
    feat.append(d['helpful']['outOf'])
    categories = set([])
    num_categories = 0
    for i in d['categories']:
        for j in i:
            if j in categories:
                continue
            categories.add(j)
            num_categories += 1
    feat.append(num_categories)
    review_words = 0
    summary_words = 0
    r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
    for w in r.split():
        w = stemmer.stem(w)
        if w in words:
            review_words += 1
    feat.append(review_words)
    punctuation_count = len([c for c in d['reviewText'] if c in punctuation])
    feat.append(punctuation_count)
    r = ''.join([c for c in d['summary'].lower() if not c in punctuation])
    for w in r.split():
        w = stemmer.stem(w)
        if w in words2:
            summary_words += 1
    feat.append(summary_words)
    return feat'''

In [7]:
prices = {}
price_by_category = {}
count_by_category = {}

for d in training_data:
    if 'price' not in d or d['itemID'] in prices:
        continue
    prices[d['itemID']] = d['price']        
    current_categories = set([])
    for i in d['categories']:
        for j in i:
            if j in current_categories:
                continue
            current_categories.add(j)
            if j in price_by_category:
                price_by_category[j] += d['price']
                count_by_category[j] += 1
            else:
                price_by_category[j] = d['price']
                count_by_category[j] = 1
                
for i in price_by_category:
    price_by_category[i] *= 1.0 / count_by_category[i]

In [8]:
words = [x[1] for x in counts[:50]]

In [9]:
def feature(d):
    feat = []
    feat.append(d['rating'])
    feat.append(d['unixReviewTime'])
    feat.append(d['helpful']['outOf'])
    if 'price' in d:
        feat.append(d['price'])
    elif d['itemID'] in prices:
        feat.append(prices[d['itemID']])
    else:
        current_categories = set([])
        total = 0
        ctr = 0
        for i in d['categories']:
            for j in i:
                if j in current_categories:
                    continue
                current_categories.add(j)
                if j not in price_by_category:
                    continue
                total += price_by_category[j]
                ctr += 1
        feat.append(total * 1.0 / ctr)
    r = ''.join([c for c in d['summary'].lower() if not c in punctuation])
    review_tokens = r.split()
    for i in words:
        if i in review_tokens:
            feat.append(1)
        else:
            feat.append(0)
    punctuation_count = len([c for c in d['reviewText'] if c in punctuation])
    feat.append(punctuation_count)
    return feat

In [10]:
X_train = []
y_train = []

for d in training_data:
    X_train.append(feature(d))
    y_train.append(d['helpful']['nHelpful'] * 1.0 / d['helpful']['outOf'])

In [11]:
#theta, residuals, rank, s = numpy.linalg.lstsq(X_train, y_train)

X_validation = []
y_validation = []

for d in validation_data:
    X_validation.append(feature(d))
    y_validation.append(d['helpful']['nHelpful'] * 1.0 / d['helpful']['outOf'])

In [12]:
model = Sequential()

model.add(Dense(output_dim=64, input_dim=len(X_train[0]), init = 'uniform'))
model.add(Activation("tanh"))
model.add(Dense(output_dim=1, init = 'uniform'))

model.compile(loss='mse', optimizer='sgd', metrics=['mean_absolute_error'])

In [ ]:
model.fit(X_train, y_train, nb_epoch=50, batch_size=32)

loss_and_metrics = model.evaluate(X_validation, y_validation, batch_size=32)

Epoch 1/50
53468/53468 [==============================] - 1s - loss: 0.1262 - mean_absolute_error: 0.2814     

In [143]:
error = 0.0

for d in validation_data:
    predicted = sum([a * b for a, b in zip(theta, feature(d))]) * d['helpful']['outOf']
    error += abs(predicted - d['helpful']['nHelpful'])
    
print error

6660.35032532


In [144]:
predictions = open("predictions_Helpful.txt", 'w')

preds = []

count = 0

for d in readGz('test_Helpful.json.gz'):
    preds.append(sum([a * b for a, b in zip(theta, feature(d))]))

for l in open("pairs_Helpful.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,i,outOf = l.strip().split('-')
    outOf = int(outOf)
    predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(preds[count] * outOf) + '\n')
    count += 1
    
predictions.close()

In [100]:
# Rating